In [ ]:
spark

In [ ]:
sc

In [ ]:
spark.sparkContext.setLogLevel("INFO")

# Pre-processando a Base de Dados

In [ ]:
from pyspark.sql.functions import *
import pandas as pd

In [ ]:
!hdfs dfs -ls -h /user/jonatas/data/

In [ ]:
!hdfs dfs -cat '/user/jonatas/data/MICRODADOS_ENEM_2021.csv' | head -3

Dada a quantidade elevada de campos do conjunto de dados, foi optado por ser feita a inferência do esquema automaticamente para agilizar análise dos dados em detrimento da otimização do espaço usado na memória. ponto de melhoria  

In [ ]:
df_dados_enem = spark.read.csv("hdfs:///user/jonatas/data/MICRODADOS_ENEM_2021.csv"
                               ,sep=";"
                               ,inferSchema="True"
                               ,header="True"
                               ,encoding="latin1"
                              )

Salvamos os dados em formato parquet para otimizar espaço e agilizar a leitura dos dados brutos caso seja nessessário futuramente

In [ ]:
#!hdfs dfs -rm -r /user/jonatas/data/MICRODADOS_ENEM_2021

In [ ]:
df_dados_enem.write.parquet("hdfs:///user/jonatas/data/MICRODADOS_ENEM_2021")

In [ ]:
!hdfs dfs -ls -h /user/jonatas/data/

In [ ]:
!hdfs dfs -du -s -h hdfs:///user/jonatas/data/MICRODADOS_ENEM_2021

# Processamento dos dados

In [ ]:
from pyspark.sql.functions import *
import pandas as pd

In [ ]:
df_dados_enem = spark.read.parquet("hdfs:///user/jonatas/data/MICRODADOS_ENEM_2021")

In [ ]:
df_dados_enem.show(1)

In [ ]:
df_dados_enem.count()

In [ ]:
df_dados_enem.printSchema()

Com auxílio do arquivo de dicionário e a análise do schema do dataframe foi possível identificar os campos e a sua relevância para a análise que pretendemos fazer. Realizando assim uma primeira filtragem dos dados,  foram removidos os campos que não são interessantes para a análise. Como a cor do caderno das provas realizadas, e o vetor das respostas e gabarito. Como dada a anatureza dos dados, o foco na análise se dará no perfil social, racial, econômico, regional e nota dos participantes, informações como qual cor de caderno eles receberam, ou quais resposta marcaram é irelevante para o estudo. O número de inscrição também não tem valor explicativo.

In [ ]:
df_tratado = df_dados_enem.drop(
    'NU_INSCRICAO',
    'CO_PROVA_CN',
    'CO_PROVA_CH', 
    'CO_PROVA_LC', 
    'CO_PROVA_MT',
    'TX_RESPOSTAS_CN',
    'TX_RESPOSTAS_CH',
    'TX_RESPOSTAS_LC',
    'TX_RESPOSTAS_MT',
    'TX_GABARITO_CN',
    'TX_GABARITO_CH',
    'TX_GABARITO_LC',
    'TX_GABARITO_MT'
)

In [ ]:
df_tratado.show(2)

In [ ]:
!hdfs dfs -rm -r /user/jonatas/data/df_tratado_1

Salvamos o dataframe com o primeiro tratamento com o índice 1

In [ ]:
df_tratado.write.parquet("hdfs:///user/jonatas/data/df_tratado_1")

In [ ]:
!hdfs dfs -du -s -h /user/jonatas/data/df_tratado_1

# Análise do Conjunto de Dados

Inicialmento foi optado por fazer uma análise voltada para Feature Selection para identificar quais variávies mais explicam a nota final do ENEM, visando o desenvolvimento de um modelo de regressão para prever essa nota final.

Sendo a nota final do ENEM definida como a média das 5 provas, as 4 objetivas e redação.

Using the largest variance criteria would be akin to feature extraction,

In [ ]:
from pyspark.sql.functions import *
import pandas as pd

In [ ]:
df_tratado = spark.read.parquet("hdfs:///user/jonatas/data/df_tratado_1")

In [ ]:
df_tratado

## Análise das variáveis

Um ponto a se destacar é que praticamente todas as nossas variáveis são qualitativas, e quando for feita a mádia das notas para compor a Nota Final que será o Target da análise, todas as variáveis restantes serão qualitativas. Para se criar um modelo de previsão em cima disso se torna necessário convertelas em variáveis dummy e isso acaretará problemas para a nossa base no estado atual que ela se encontra como veremos adiante.

O principal problema se mantém sendo o tamanho da base de dados e a quantidade de colunas, o que frustrou tentativas anteriores de uma manipulação como dataframe do pandas, sendo assim necessário uma análise mais minunciosa da base e tratamento como dataframe do spark para a remoção de informação desnecessária.

Como sabemos que estamos analisando os dados do enem 2021, sendo assim não precisamos da coluna de ano. 

Como Além do Distrito Federal e do distrito insular de Fernando de Noronha, o Brasil tem 5.568 municípios, foi optado fazer a análise somente a nível estadual pois dado que todas nossas variáveis são qualitativas e assim  atarpalharia o modelo de dummies, removendo assim as colunas de código do município e nome do município. 

Mantendo somente codigo da UF da escola onde foi cursado o EM e a da realização da prova. 

As colunas de presença nas provas objetivas e status da redação não são necessárias pois só serão considerados os participantes presentes em todas as provas e que tenham feito pontos, a remoção dos que não se encaixam nessa categoria será feita com a remoção dos NaN da Coluna a ser criada NF_Enem. 

As notas das competências da redação também não serão avaliadas individualmente e nem o tipo de língua estrangeira selecionada.

In [ ]:
df_tratado = df_tratado.drop(
    #'CO_PROVA_CN','CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT',
    #'TX_RESPOSTAS_CN','TX_RESPOSTAS_CH','TX_RESPOSTAS_LC','TX_RESPOSTAS_MT',
    #'TX_GABARITO_CN','TX_GABARITO_CH','TX_GABARITO_LC','TX_GABARITO_MT',
    #'NU_INSCRICAO',
    
    'NU_ANO',
    'CO_MUNICIPIO_ESC','NO_MUNICIPIO_ESC','SG_UF_ESC','TP_SIT_FUNC_ESC',
    'CO_MUNICIPIO_PROVA','NO_MUNICIPIO_PROVA', 'SG_UF_PROVA',
    'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC','TP_PRESENCA_MT',
    #'NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT', 'NU_NOTA_REDACAO',
    'TP_STATUS_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 
    'TP_LINGUA'
)

In [ ]:
#!hdfs dfs -rm -r /user/jonatas/data/df_tratado_2

In [ ]:
df_tratado.write.parquet("hdfs:///user/jonatas/data/df_tratado_2")

In [ ]:
!hdfs dfs -du -s -h /user/jonatas/data/df_tratado_2

# Manipulação dos dados

In [ ]:
from pyspark.sql.functions import *
import pandas as pd

In [ ]:
df_tratado = spark.read.parquet("hdfs:///user/jonatas/data/df_tratado_2")

In [ ]:
df_tratado = df_tratado.withColumn(
    'NF_ENEM',
    round(
    df_tratado.NU_NOTA_CN +
    df_tratado.NU_NOTA_CH +
    df_tratado.NU_NOTA_LC +
    df_tratado.NU_NOTA_MT +
    df_tratado.NU_NOTA_REDACAO
    ) / 5
).drop('NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT', 'NU_NOTA_REDACAO')

In [ ]:
df_tratado.count()

In [ ]:
#df_tratado = df_tratado.na.drop(subset=['NF_ENEM'])
df_tratado = df_tratado.na.drop()

Uma vez que selecionamos as colunas relevantes para nossa análise, um último passo é selecionar apenas os participantes que não tem nenhuma informação faltante.

In [ ]:
df_tratado.count()

In [ ]:
df_tratado.show(10)

In [ ]:
!hdfs dfs -rm -r /user/jonatas/data/df_tratado

In [ ]:
df_tratado.write.parquet("hdfs:///user/jonatas/data/df_tratado_final")

In [ ]:
!hdfs dfs -du -s -h /user/jonatas/data/df_tratado_final

Conseguimos reduzir bastante o tamanho da base de dados

# Análise com Pandas

In [ ]:
from pyspark.sql.functions import *
import pandas as pd

In [ ]:
df = pd.read_parquet("hdfs:///user/jonatas/data/df_tratado_final")

In [ ]:
len(df.index)

In [ ]:
print(df.columns.tolist())

Para realizarmos a análise precisamos da criação de dummys

In [ ]:
dumies_list = pd.Series([
    'TP_FAIXA_ETARIA', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU',
    'TP_ESCOLA', 'TP_ENSINO', 'IN_TREINEIRO', 'CO_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'CO_UF_PROVA', 
    'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006', 'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 
    'Q013', 'Q014', 'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022', 'Q023', 'Q024', 'Q025'
])

In [ ]:
df_analise = pd.get_dummies(df, columns=dumies_list)

In [ ]:
df_analise

In [ ]:
print(df_analise.columns.tolist())

In [ ]:
df_analise.to_parquet('hdfs:///user/jonatas/data/df_tratado_final_dummys')

In [ ]:
!hdfs dfs -du -s -h /user/jonatas/data/df_tratado_dummys

## Feature Selection

A primeira análise proposta foi a de selecionar as variávies que mais tem influência para nosso target e somente posteriormente implementar o modelo de previsão

In [ ]:
from pyspark.sql.functions import *
import pandas as pd

In [ ]:
df_analise = pd.read_parquet("hdfs:///user/jonatas/data/df_tratado_final_dummys")

In [ ]:
lista = df_analise.corr()["NF_ENEM"]
#lista_abs = df_teste.corr()["NF_ENEM"].abs().sort_values(ascending = False)

In [ ]:
!hdfs dfs -ls -h hdfs:///user/jonatas/data

In [ ]:
#lista_pros = lista.sort_values(ascending = False)

In [ ]:
#print(lista_pros.to_string())

In [ ]:
#lista_cons = lista.sort_values(ascending = True)

In [ ]:
#print(lista_cons.to_string())

In [ ]:
lista_abs = lista.abs().sort_values(ascending = False)

In [ ]:
#print(lista_abs[1:126].to_string())

In [ ]:
regressors = lista_abs.loc[lista_abs >= 0.15]

In [ ]:
regressors = regressors.index[1:].tolist()

In [ ]:
regressors

In [ ]:
df = df_analise[['NF_ENEM'] + regressors]

In [ ]:
df

In [ ]:
df.to_parquet("hdfs:///user/jonatas/data/df_tratado_final_dummys_selected/")

In [ ]:
!hdfs dfs -ls -h /user/jonatas/data/

# Implementação do modelo de previsão

In [1]:
from pyspark.sql.functions import *
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn import linear_model

In [2]:
df = pd.read_parquet("hdfs:///user/jonatas/data/df_tratado_final_dummys_selected")

In [ ]:
#rows = df.shape[0]
#rows

In [ ]:
#df_teste  = df.sample(np.int(0.02*rows))

In [3]:
# Dividir a Base de teste entre váriáveis dependendes e independente

X, y = df.iloc[:,1:].values, df.iloc[:,0].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.02, random_state=42)

In [ ]:
print (X[0:10])

## Lasso

In [8]:
from sklearn import linear_model

#Definir modelo
model = linear_model.Lasso(alpha=0.1)

#Treinar modelo
model.fit(X_train, y_train)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [9]:
#Testar Modelo
model.score(X_test,y_test)

0.36180393519140785

## Linear

In [10]:
from sklearn.linear_model import LinearRegression

In [ ]:
 X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
>>> # y = 1 * x_0 + 2 * x_1 + 3
>>> y = np.dot(X, np.array([1, 2])) + 3
>>> reg = LinearRegression().fit(X, y)
>>> reg.score(X, y)
1.0
>>> reg.coef_
array([1., 2.])
>>> reg.intercept_
3.0...
>>> reg.predict(np.array([[3, 5]]))
array([16.])